In [2]:
# install neccery libs
!pip install pandas numpy scikit-learn nltk gensim

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import KeyedVectors

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords

# نمایش چند کلمه stopword
print(stopwords.words('english')[:5])

['a', 'about', 'above', 'after', 'again']


In [6]:
# خواندن دیتاست با ستون‌های مناسب
file_path = 'training.1600000.processed.noemoticon.csv'
column_names = ['sentiment', 'id', 'date', 'query', 'user', 'text']

df = pd.read_csv(
    file_path,
    header=0,  # سطر اول header هست
    encoding='latin1',
    names=column_names,
    low_memory=False
)

# فیلتر فقط برای 0 و 4
df = df[df['sentiment'].isin([0, 4])]

print("✅ تعداد داده‌ها:", len(df))
print("توزیع کلاس‌ها:")
print(df['sentiment'].value_counts())

✅ تعداد داده‌ها: 1048572
توزیع کلاس‌ها:
sentiment
0    799996
4    248576
Name: count, dtype: int64


In [7]:
from nltk.corpus import stopwords
import re

# تنظیم stop words + کلمات اضافی
stop_words = set(stopwords.words('english'))
extra_stopwords = {'rt', 'user', 'url', '…', 'amp', 'via', 'http', 'https'}
all_stopwords = stop_words.union(extra_stopwords)

# تابع پیش‌پردازش بهتر شده
def preprocess_text_improved(text):
    text = text.lower()
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in all_stopwords]
    return filtered_tokens

# اعمال پیش‌پردازش
df['tokens'] = df['text'].apply(preprocess_text_improved)

In [8]:
import gensim.downloader as api

# دانلود مدل "glove-twitter-25" - حدود 86 مگابایت
embedding_model = api.load("glove-twitter-25")

print("✅ مدل embedding با موفقیت دانلود و بارگذاری شد!")

[==================================================] 100.0% 104.8/104.8MB downloaded
✅ مدل embedding با موفقیت دانلود و بارگذاری شد!
